In [6]:
import re 
import sqlite3

""" 
Функция для единоразового создания таблицы

"""
def create_table():
    conn = sqlite3.connect("laborat6.db")
    c.execute("""CREATE TABLE article
                (Author, Title, Journal, Year)
          """)
""" 
Функция для заполнения созданной таблицы полученными данными

"""
def insrt705(Author,Journal,Title,Year):
    c = conn.cursor()
    purchases = [(Author, Journal, Title, Year)]
    c.executemany("INSERT INTO Article VALUES (?,?,?,?)",purchases)
    conn.commit()
""" 

Функция для преобразования bib-файла с данными научных публицаций к формату ГОСТ 7.0.5 при помощи регулярных выражений.

"""
def gost705(art, conn):
    ls =['Author','Journal','Title','Year']
    art_tmp = {}
    for x in ls:
        if x in art:
            art_tmp[x]= art.get(x)
        else:
            art_tmp[x] = ''
    art_tmp['Author'] = art_tmp['Author'].replace(' and ',', ')
    insrt705(**art_tmp)
""" 
Функция для вывода данных статей по запрашиваемой фамилии автора

"""
def print_table(Surname, conn):
    c = conn.cursor()
    user = ["%"+Surname+"%",]
    c.execute("SELECT * FROM article Where Author Like ?", user)
    print(c.fetchall())
    

openf = open(r'C:\Users\Artem Altyshev\Desktop\biblio1.bib', mode='r', encoding = 'utf-8') 
data = openf.read()

reg1 = re.compile(r'\@(?P<type>\w+){(?P<lable>.+?),(?P<body>.+?)}\s*}', re.DOTALL)
reg2 = re.compile(r'(\w+)\s*=\s*{(.+?)}', re.DOTALL)
iterat = re.finditer(reg1,data)
art_list=[]
for i in iterat:
    st = i.group('body') + '}'
    body = reg2.findall(st)
    art = {
           'type':i.group('type'),
           'lable':i.group('lable')}
    for it in body:
        art[it[0]] = it[1]
    art_list.append(art)
    
conn = sqlite3.connect('laba6.db') 
c = conn.cursor()
#create_table() - создаем таблицу
for art in art_list:
    gost705(art, conn)

print('Введите фамилию автора:')
author=(input())
print_table(author, conn)

Введите фамилию автора:
Черепанов
[(' Смородин, Б. Л., Черепанов, И. Н. ', 'X Всероссийский съезд по фундаментальным проблемам теоретической и прикладной механики, 24-30 августа ', 'Устойчивость адвективного течения магнитной жидкости в горизонтальном канале', 'Пермь. — 2010'), ('Смородин, Б. Л., Черепанов, И. Н. ', '', 'Устойчивость адвективного течения магнитной жидкости в горизонтальном канале', '2011'), (' Смородин, Б. Л., Черепанов, И. Н., Мызникова, Б. И. ', 'XVII Зимняя школа по механике сплошных сред, 28 февраля -- 3 марта. Тезисы докладов ', 'Конвекция стратифицированной коллоидной бинарной смеси', 'Пермь. — 2011'), (' Черепанов, И. Н., Смородин, Б. Л.', 'Материалы краевой научно-практической конференции студентов, аспирантов и молодых ученых «Физика для Пермского края»', 'Волновые режимы течения коллоидной смеси при наличии термодиффузии', 'Пермь.— 2012'), ('Черепанов, И. Н., Смородин, Б. Л. ', 'Тезисы докладов Всероссийской конференции молодых ученых «Неравновесные процессы 